In [1]:
#################################################
# Section 1. 모델 학습과 메타데이터 로그 기록 남기기 위한 준비
#################################################
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# dataset 로드
X, y = datasets.load_iris(return_X_y=True)

# 데이터를 학습용과 테스트요으로 분리
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 모델 하이퍼파라미터 설정
params = {
    "solver": "lbfgs",
    "max_iter": 1000,
    "multi_class": "auto",
    "random_state": 8888,
}

# 모델 학습
lr = LogisticRegression(**params)
lr.fit(X_train, y_train)

# 테스트 dataset 예측
y_pred = lr.predict(X_test)

# metrics 계산
accuracy = accuracy_score(y_test, y_pred)


/home/mlstudio/.local/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


In [2]:
#################################################
# Section 2. 모델학습정보 로그와 산출물(artifacts) MLFlow에 저장
#################################################
import mlflow
from mlflow.models import infer_signature
from mlstudio_sdk.config import Config

# MLStudio 환경설정보 로드
config = Config()

# [필수] MLFlow Tracking 서버 uri 정보 설정
mlflow.set_tracking_uri(uri=config.get_mlflow_tracking_uri())

# [선택] MLFlow 사용자 ID와 비밀번호
# 값 입력하지 않으면 환경변수에 저장된 사용자 id, password값 전달
login_id='test1'
login_pwd='test1'

# [필수] 모델학습정보 로그가 저장될 Experiment가 없다면 생성
experiment_name='실험명 한글 사용이 되는건가?' # Experiment 이름
# create_experiment_if_not_exists(login_id, login_pwd, experiment_name) # Experiment가 없다면 새성한다.
experiment = mlflow.set_experiment(experiment_name)

# [선택] 모델정보가 저장될 Registered model이 없다면 생성
# Registered model에 등록하지 않을경우 
registered_model_name='모델 레지스트리 한글 사용이 되는건가?' # Registered model 이름
create_registered_model_if_not_exists(login_id, login_pwd, name=registered_model_name) # Registered model가 없다면 생성한다.

# 활성화된 MLFlow run이 있다면 끝내기
mlflow.end_run()

# MLflow run 시작
with mlflow.start_run():
    # 하이퍼파라미터 로그 기록
    mlflow.log_params(params)

    # loss metric 로그 기록
    mlflow.log_metric("accuracy", accuracy)

    # 향후 현재 MLFlow run정보 상기 위한 tag 붙이기
    mlflow.set_tag("Training Info", "Basic LR model for iris data")

    # 모델 시그니처를 추론
    signature = infer_signature(X_train, lr.predict(X_train))

    # 모델정보 로그와 산출물(artifact) MLFlow에 기록
    model_info = mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path="iris_model",
        signature=signature,
        input_example=X_train,
        registered_model_name=registered_model_name,
    )


MlflowException: Cannot set a deleted experiment '실험명 한글 사용이 되는건가?' as the active experiment. You can restore the experiment, or permanently delete the experiment to create a new one.

In [38]:
#################################################
# Load the model back for predictions as a generic Python Function model
#################################################
loaded_model = mlflow.pyfunc.load_model(model_info.model_uri)

predictions = loaded_model.predict(X_test)

iris_feature_names = datasets.load_iris().feature_names

result = pd.DataFrame(X_test, columns=iris_feature_names)
result["actual_class"] = y_test
result["predicted_class"] = predictions

result[:4]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),actual_class,predicted_class
0,6.1,2.8,4.7,1.2,1,1
1,5.7,3.8,1.7,0.3,0,0
2,7.7,2.6,6.9,2.3,2,2
3,6.0,2.9,4.5,1.5,1,1
